# Wpowadzenie do deep learning w bibliotece Flux.jl

## Przykład

Aby w  zrozumieć sposób pracy z Fluxem warto rozpatrzeć prosty przykład. Zajmiemy się przetwarzaniem języka naturalnego - zbudujemy model zdolny do generowania składnej wypowiedzi w języku polskim.

Wyjściowe założenie jest takie, że wytrenujemy sieć neuronową, która będzie estymowała prawdopodobieństwo wystąpienia danego znaku w ciągu na podstawie poprzedzających go znaków w sekwencji ([<b>Character-Level Language Model</b>](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)).

Zbiórem na którym będziemy pracowali jest <i>W poszukiwaniu straconego czasu</i> Marcela Prousta. 

[![](https://upload.wikimedia.org/wikipedia/commons/b/b8/Marcel_Proust_1895.jpg)](https://pl.wikipedia.org/wiki/Marcel_Proust)

>(...) matka widząc, że mi jest zimno, namówiła mnie, abym się napił wbrew zwyczajowi trochę herbaty. Odmówiłem zrazu; potem, nie wiem czemu, namyśliłem się. Posłała po owe krótkie i pulchne ciasteczka zwane magdalenkami, które wyglądają jak odlane w prążkowanej skorupie muszli. I niebawem (...) machinalnie podniosłem do ust łyżeczkę herbaty, w której rozmoczyłem kawałek magdalenki. Ale w tej samej chwili, kiedy łyk pomieszany z okruchami ciasta dotknął mego podniebienia, zadrżałem, czując, że się we mnie dzieje coś niezwykłego. Owładnęła mną rozkoszna słodycz (...). Sprawiła, że w jednej chwili koleje życia stały mi się obojętne, klęski jako błahe, krótkość złudna (...). Cofam się myślą do chwili, w której wypiłem pierwszą łyżeczkę herbaty (...). I nagle wspomnienie zjawiło mi się. Ten smak to była magdalenka cioci Leonii.(...)

Zanim jednak zaczniemy wprowadźmy odrobinę teorii stojącej za tym zagadnieniem:

### Rekurencyjne sieci neuronowe (Recurrent neural networks)

- Charakterystyczną cechą tego typu sieci jest to, że pozwalają one na istnienie wewnątrz grafu cykli skierowanych.
- Oznacza to, że informacja wewnątrz takiej sieci nie musi płynąć tylko w jednym kierunku - neurony leżące na tej samej warstwie także mogą przesyłać sobie wzajemnie dane:

[![](http://karpathy.github.io/assets/rnn/diags.jpeg)](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)

Dzięki tej właściwości RNN doskonale nadają się do budowy interesującego nas modelu: 

[![](http://karpathy.github.io/assets/rnn/charseq.jpeg)](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)

### Long short-term memory

Problemem na który można natrafić w przypadku korzystania z RNN jest pamięć takiej sieci. Gdy odległość pomiędzy aktualnym a poprzedzającymi go węzłami, które niosą za sobą kluczową informację jest niewielka, sieć jest w stanie efektywnie je wykorzystać:

[![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-shorttermdepdencies.png)](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)

Problem się pojawia gdy ta odległość jest duża - wtedy kluczowe informacje po prostu znikają w szumie:

[![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-longtermdependencies.png)](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)

Wtedy też, warto zastosować sieć LSTM, która ze względu na swoją architekturę jest w stanie odpowiednio filtrować informację i wykorzystawać je nawet wtedy, gdy ich źródło jest znacznie oddalone od aktualnego neuronu:

[![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png)](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)

Przejdźmy teraz do implementowania modelu za pomocą Fluxa:

### Implementacja

In [1]:
using Flux
using Flux: onehot, argmax, chunk, batchseq, throttle, crossentropy
using StatsBase: wsample
using Base.Iterators: partition
using BSON

Pierwszym krokiem jest oczywiście odpowiednie przygotowanie danych na których będziemy pracowali:

In [2]:
text = collect(read("w_poszukiwaniu.txt",String));
alphabet = [unique(text)..., '_'];

Następnie kodujemy zmienne jakościowe:

In [3]:
text = map(ch -> onehot(ch, alphabet), text);
stop = onehot('_', alphabet);

In [4]:
N = length(alphabet);
seqlen = 100;
batch_size = 32;

In [5]:
Xs = collect(partition(batchseq(chunk(text, batch_size), stop), seqlen));
Ys = collect(partition(batchseq(chunk(text[2:end], batch_size), stop), seqlen));

In [6]:
m = Chain(
    LSTM(N, 128),
    Dropout(0.3), 
    LSTM(128, 128),
    Dropout(0.3),
    Dense(128, N),
    softmax)

function loss(xs, ys)
  l = sum(crossentropy.(m.(xs), ys))
  #Flux.reset!(m)
  return l
end

opt = ADAM(0.001)


function sample(m, alphabet, len; temp = 1)
  Flux.reset!(m)
  buf = IOBuffer()
  c = rand(alphabet)
  for i = 1:len
    write(buf, c)
    c = wsample(alphabet, m(onehot(c, alphabet)))
  end
  return String(take!(buf))
end

#evalcb = () -> @show loss(Xs[5], Ys[5])

evalcb = function ()
    @show loss(Xs[5], Ys[5])
    println(sample(m, alphabet, 100))
end

#4 (generic function with 1 method)

In [7]:
sample(m, alphabet, 50)

"½—péâHtQ1à2ą4m)O\rMS,àpїG'«5â!…HäćŚT.öNKŚ8Ê5öu-3xĆf"

In [8]:
@info("Beginning training loop...")
best_ls = Inf
last_improvement = 0
for epoch = 1:10
    @info "Epoch: $epoch"
    global best_ls, last_improvement
    Flux.train!(loss, params(m), zip(Xs, Ys), opt, cb=throttle(evalcb, 240))
    ls = loss(Xs[5], Ys[5])
    if ls <= best_ls
        @info "New best result: $ls"
        BSON.@save "char_model.bson" m
        best_ls = ls
        last_improvement = epoch
    end
    if epoch - last_improvement >= 3
        @warn(" -> We're calling this converged.")
        break
    end
end

┌ Info: Beginning training loop...
└ @ Main In[8]:1
┌ Info: Epoch: 1
└ @ Main In[8]:5


loss(Xs[5], Ys[5]) = 474.34888f0
vKbHuŹ7Koé
n(”łBłnù8”ŃiWFùtx*ŹÊїüèYŚ8äGAQStFOÓ15É:3ö°jä5îŁ°5Q,êP
loss(Xs[5], Ys[5]) = 325.04382f0
loeen,  sre  aęiwnw g apzeó  nŹyewacrwso pązeieąkaak !imto meąunocesp
loss(Xs[5], Ys[5]) = 305.06128f0
ni  kałęoąk nyurwełi cca,z dier? bo bjdepęanjpysko    wjrt
loss(Xs[5], Ys[5]) = 290.61182f0
otén rzkrrenę manieaeo I 5yrnnt sjach0ot
loss(Xs[5], Ys[5]) = 278.1952f0
?-Ogybczmącm ddaną jytro zrzeję i nidczlęługrerzugr ac ky ś ópsie snęcie biebeksh k wamkzac,y  óbłeo
loss(Xs[5], Ys[5]) = 271.22467f0
ż d, e ę ma ośta subę  zławera kosczjokowzdę die a wynlona zowtegośnycie etaCw ,,
loss(Xs[5], Ys[5]) = 266.5139f0
èD°litakam noneąjąj atek niesikswuą łypł m ż naTsieznnąachch,iadzią pnąpnałeH daananz enidłe kaułobn
loss(Xs[5], Ys[5]) = 262.8068f0
T8ç9u umakuzbza wwą zą, as, tatszemniedzytotym zielui, dłem mian ztnmeją dróregosa , kenińc uła dowf


┌ Info: New best result: 259.97333
└ @ Main In[8]:10
┌ Info: Epoch: 2
└ @ Main In[8]:5


loss(Xs[5], Ys[5]) = 259.9568f0
!kŻ,dza , O kij nuzadrzad woła zobzo todo konomz mudzamaglofiłudu, wisięji k gy mureszydłojeo odtaj 
loss(Xs[5], Ys[5]) = 255.9093f0
»inù,v , k jenam orjienrsiynią w rały matań,c yiwłeroylęge, zmi:.
T
W jegwałą Pany mniesięcy, taj
loss(Xs[5], Ys[5]) = 253.64703f0
Ź_s,jyłeujnj li speznsczeż mimow krzeinierni słe èślicildza, wsymó mst, mego dablawwalżą by łoczem u
loss(Xs[5], Ys[5]) = 251.50319f0
9)Jyoyny wiumaznalo onte sze nousny ly wienen drośłaogor'ć cy Tos k przy, wik przę gor wa mien wamsm
loss(Xs[5], Ys[5]) = 247.31192f0
DL„rwęną, wie: kruw chy. Dać, jejesci miż ślóo.

Ią, (wrztłe rny uwunoj krojezz, ku Mi— an cziecza
loss(Xs[5], Ys[5]) = 245.38797f0
kŚ­c oa niolem, sięże, dóawie,, jego pas proewo re kazy wosęt zazopłeło mośwu t onszesbab na włe ban
loss(Xs[5], Ys[5]) = 243.8592f0
Àż riźłć wy kaj moć. Naa wpaż koś, z,, nrarwechedb, miemem niem ka lebsie przeocłe cichchm gobkie 
loss(Xs[5], Ys[5]) = 242.0263f0
jLdcż,c sle ich Fi niąwanłynkt” nim, twr

┌ Info: New best result: 242.30338
└ @ Main In[8]:10
┌ Info: Epoch: 3
└ @ Main In[8]:5


...

loss(Xs[5], Ys[5]) = 228.83557f0
/Pdowreru, tonych”. nagnyerpadkę lozylto kobowanego możejaj nieżałem, bie teo optmietzenwje aj mawte
loss(Xs[5], Ys[5]) = 229.0768f0
8pbj dawa prw! żnlnieciny oddzia pomiynarwowtomym „— Niażemiła waroniniej wi nie miedząkie powiaciwa
loss(Xs[5], Ys[5]) = 228.17899f0
êpnkJie, jąc żegrózrosjej kosię zdawdiwedbla ziczawzychakser siąwąłerrzecczaką sim nana jak cońgało,
loss(Xs[5], Ys[5]) = 227.64122f0
bjkziego a kie zusprają w  bzy kłoraw sólrachem wowem deczejie gliptów tak, zzzywy mólarz tzytrohiwe
loss(Xs[5], Ys[5]) = 227.1599f0
qkięnry sawszedałem i podzie mikcy an nieki pała josłoś i przeciem owie Lota przy, chaniała stłykccu
loss(Xs[5], Ys[5]) = 226.51381f0
Êńżuwi wałałbywa sławką stubuszyczarucąsistachościec mi b tłu bu, która przyłogzieńszkej to apzu pna
loss(Xs[5], Ys[5]) = 226.44017f0
:/pazwzgiegi łyłem a ek ttetodeńch, ąwozcty naznie o spon wem  pierłnna którymy kapepich z i A mo ni
loss(Xs[5], Ys[5]) = 226.52805f0
'Zijąty udnertech kj rot, ntole

┌ Info: New best result: 226.14456
└ @ Main In[8]:10
┌ Info: Epoch: 9
└ @ Main In[8]:5


loss(Xs[5], Ys[5]) = 226.27109f0
szŁpewimusię jad znawdalniadzujopowaj konichonkuf-Cy łały była”naśsky posi mię niełem, żaczyków niąt
loss(Xs[5], Ys[5]) = 225.94597f0
Ênaeillwistyte-ść pokrajtejacze polnialat, której chciw i , wiagamie misusstą wiwodym stw” go zojawd
loss(Xs[5], Ys[5]) = 226.40259f0
9m—Bbolsustwobodziąrobigda znowałbo pugrobyrogą:


W znhojgaż. Ali na.

—, balne przew spon śwy
loss(Xs[5], Ys[5]) = 226.96422f0
ńchrachycią otrwiowniobly tary, tłazlkatarzoją ści lalumieczy. A — na ciedzie jękarospakrórowie lurs
loss(Xs[5], Ys[5]) = 227.09627f0
CnazckincycierdłwewŻone zresodziejtowjeżałeicię rzerdwne bie mrieńć jej sło zoe innał momkana wydzo 
loss(Xs[5], Ys[5]) = 226.82191f0
Ininodutnobiekazu uczegałimnd ma wyrdze,b dy się inałem seu (niem s nanią pość sprowaj, to żajawszia
loss(Xs[5], Ys[5]) = 227.26387f0
mpę.winyśstumi marwzcszuzrowo cie judzięt jej sopo zak dudziaszę;
Iodzie sato la, puczabnana zi w b
loss(Xs[5], Ys[5]) = 228.00194f0
îkaa. tassze nant odsęprzabesola, w

┌ Info: Epoch: 10
└ @ Main In[8]:5


loss(Xs[5], Ys[5]) = 227.31454f0
—zylie tesczem kdzcaniemiego, alomo ni kóbena die dow było skiem buby morzeczow wypymy zroją, wiez B
loss(Xs[5], Ys[5]) = 227.25938f0
8isalmutttode scie warzyśszco modzeczała — te bowawych nie ym.

— Alerogać, glacomakny islSoi wdak
loss(Xs[5], Ys[5]) = 227.81526f0
Rt.ą) — panio mieczyjchucktupntem prówaczęjoczenie z a dlowe w dral pank żegaciacją Alekeiesztazie ż
loss(Xs[5], Ys[5]) = 226.90192f0
;iovikzudnie niny w. Tektajupi do czy się zk wył snen, go któiesącej: kię ęów Vako koskę” Oj zrociee
loss(Xs[5], Ys[5]) = 226.89091f0
Yenoapa szę odziwalyjertrarowaniej i *cielijeąrarpie łanej n sącin. Ro, przywko ddasmych by się, ber
loss(Xs[5], Ys[5]) = 226.80936f0
iiekórórygoganysch. Tożą się do „Siennąc dzy ominnanowcią, Rów, łyćy poduchakwał nie przyjaćał  i Sn


In [8]:
BSON.@load "char_model.bson" m

In [11]:
sample(m, alphabet, 50)

")pzzewsztejiża prtej pamanieś cnią, dujów mygogorr"

## Dodatkowa praca domowa

Zmodyfikuj kod tak, aby poprawić jakość generowanego tekstu.